In [55]:
#!pip install simpletransformers

In [3]:
import estnltk
import os
import time

from estnltk import Text
import re
from estnltk.converters import text_to_json, json_to_text

from estnltk_core.layer.span_operations import conflict
import pandas as pd

# Import adapted words tokenization
from words_tokenization import preprocess_words # vaja words_tokenization.py

# Import adapted sentence tokenizer
from sentence_tokenization import sentence_tokenizer # vaja sentence_tokenization.py
from sentence_tokenization import postfix_sentence_breaks_inside_parentheses # vaja sentence_tokenization.py

from estnltk_core.layer_operations import extract_sections

import logging
from simpletransformers.ner import NERModel, NERArgs

INFO:config.py:54: PyTorch version 2.5.1+cu124 available.


In [2]:
OutputFolder = 'data/output4'

In [7]:
#output2 -> output3, vana words,sentences layer maha ja uus peale
for name in os.listdir('data/output2'):
    with open(os.path.join('data/output2', name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    #print(name, len(text_import.words), len(text_import.sentences))
    #print(text_import.layers)
    text_import.pop_layer('sentences')
    text_import.pop_layer('words')
    #print(text_import.layers)

    preprocess_words( text_import )
    sentence_tokenizer.tag( text_import )
    postfix_sentence_breaks_inside_parentheses( text_import, doc_name='' )
    #print(len(text_import.words), len(text_import.sentences))

    text_to_json(text_import, file=('data/output3' + "/" + name))

In [9]:
#output3 -> output4, fix some mistakes
for name in os.listdir('data/output3'):
    with open(os.path.join('data/output3', name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    if name == '1934-10-15_manual_annotated.json':
        #htusegümnaasiumi -> õhtusegümnaasiumi
        print(text_import.ne_gold_a[642], text_import.ne_gold_a[642].start)
        text_import.ne_gold_a.remove_span(text_import.ne_gold_a[642])
        text_import.ne_gold_a.add_annotation( text_import.words[7005].base_span, {'tag': 'ORG'})
        print(text_import.ne_gold_a[642], text_import.ne_gold_a[642].start)
    
    if name == '1935-09-30_manual_annotated.json':
        #Rosenfeldt -> J.Rosenfeldt
        print(text_import.ne_gold_a[431], text_import.ne_gold_a[431].start)
        text_import.ne_gold_a.remove_span(text_import.ne_gold_a[431])
        text_import.ne_gold_a.add_annotation( text_import.words[6679].base_span, {'tag': 'PER'})
        print(text_import.ne_gold_a[431], text_import.ne_gold_a[431].start)

        #majõevaheline plats -> Emajõevaheline plats
        print(text_import.ne_gold_a[597], text_import.ne_gold_a[597].start)
        text_import.ne_gold_a.remove_span(text_import.ne_gold_a[597])
        text_import.ne_gold_a.add_annotation( (54117, 54137), {'tag': 'LOC'})
        print(text_import.ne_gold_a[597], text_import.ne_gold_a[597].start)
    
    text_to_json(text_import, file=(OutputFolder + "/" + name))

Span('htusegümnaasiumi', [{'tag': 'ORG'}]) 39589
Span('õhtusegümnaasiumi', [{'tag': 'ORG'}]) 39588
Span('Rosenfeldt', [{'tag': 'PER'}]) 39097
Span('J.Rosenfeldt', [{'tag': 'PER'}]) 39095
Span('majõevaheline plats', [{'tag': 'LOC'}]) 54118
Span('Emajõevaheline plats', [{'tag': 'LOC'}]) 54117


In [15]:
def next_tag(text_import, tagnr):
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }
    
    tagnr += 1
    
    while True:
        if tagnr >= len(text_import.ne_gold_a):
            return -1
        if text_import.ne_gold_a[tagnr].tag in tags:
            return tagnr
        else:
            tagnr += 1

def process_file(file_name):
    with open(os.path.join(OutputFolder, file_name), "r", encoding="utf-8") as file:
        content = file.read()
    
    text_import = json_to_text(json_text=content)
    
    data = []
    tag_nr = next_tag(text_import, -1)
    sen_nr = 0
    
    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }

    # Process each word
    for word in text_import.words:
        # Update sentence ID if needed
        if sen_nr < len(text_import.sentences) and not conflict(text_import.sentences[sen_nr], word):
            sen_nr += 1

        if tag_nr != -1:
            tag = text_import.ne_gold_a[tag_nr]
            
            # If word is before the current entity
            if word.end <= tag.start:
                # Add as regular word
                data.append([file_name, sen_nr, word.text, "O"])

            if word.start == tag.start:
                normalized_tag = tags.get(tag.tag, tag.tag)
                data.append([file_name, sen_nr, word.text, f"B-{normalized_tag}"])
            # If word is part of the current entity
            elif conflict(tag, word):
                normalized_tag = tags.get(tag.tag, tag.tag)
                data.append([file_name, sen_nr, word.text, f"I-{normalized_tag}"])
                
            # If word is after the current entity
            elif word.start >= tag.end:
                tag_nr = next_tag(text_import, tag_nr)
                
                if tag_nr != -1:
                    tag = text_import.ne_gold_a[tag_nr]

                    if word.start == tag.start:
                        data.append([file_name, sen_nr, word.text, f"B-{normalized_tag}"])
                    else:
                        data.append([file_name, sen_nr, word.text, "O"])
                else:
                    data.append([file_name, sen_nr, word.text, "O"])
        else:
            data.append([file_name, sen_nr, word.text, "O"])
    
    return data

def process_directory_to_dict(output_folder):
    all_data = []
    
    for name in os.listdir(output_folder):
        all_data.extend(process_file(name))

    data = pd.DataFrame(all_data, columns=["filename", "sentence_id", "words", "labels"])
    return data

data = process_directory_to_dict('data/output4')
data

,filename,sentence_id,words,labels
0,1922-04-24_manual_annotated.json,0,Tartu,B-ORG
1,1922-04-24_manual_annotated.json,0,linnawolikogu,I-ORG
2,1922-04-24_manual_annotated.json,0,korraline,O
3,1922-04-24_manual_annotated.json,0,koosolek,O
4,1922-04-24_manual_annotated.json,0,esmaspäewal,O
...,...,...,...,...
51241,1941-01-03_manual_annotated.json,42,.,O
51242,1941-01-03_manual_annotated.json,43,Ärakiri,O
51243,1941-01-03_manual_annotated.json,43,õige,O
51244,1941-01-03_manual_annotated.json,43,:,O


In [17]:
#Testing if every word is present in the dataframe
for name in os.listdir(OutputFolder):
    with open(os.path.join(OutputFolder, name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    print(len(data[data["filename"] == name]) == len(text_import.words), len(data[data["filename"] == name]), len(text_import.words))

True 2219 2219
True 7236 7236
True 12795 12795
True 8981 8981
True 11739 11739
True 7585 7585
True 691 691


In [19]:
#Testing if correct numbers of tags are in the dataset and if tehre are any mismatches
for name in os.listdir(OutputFolder):
    with open(os.path.join(OutputFolder, name), "r", encoding="utf-8") as file:
        content = file.read()
    text_import = json_to_text(json_text=content)

    tags = {
        "PER": "PER",
        "LOC": "LOC",
        "ORG": "ORG",
        "ORG_POL": "ORG",
        "ORG_GPE": "ORG",
    }
    
    starting_data = [[i.text, tags[i.tag], i.start] for i in text_import.ne_gold_a if i.tag in list(tags.keys())]
    print(
        len(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))]) == len(starting_data),
        len(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))]),
        len(starting_data),
        name
    )
    print('----------')
    for nr, (idx, row) in enumerate(data[(data["filename"] == name) & (data["labels"].str.startswith('B-'))].iterrows()):
        if starting_data[nr][0].startswith(row['words']) == False:
            print(nr, starting_data[nr], row, idx)

True 166 166 1922-04-24_manual_annotated.json
----------
True 495 495 1927-03-28_manual_annotated.json
----------
True 723 723 1932-01-25_manual_annotated.json
----------
True 527 527 1934-10-15_manual_annotated.json
----------
True 562 562 1935-09-30_manual_annotated.json
----------
True 323 323 1936-09-07_manual_annotated.json
----------
True 37 37 1941-01-03_manual_annotated.json
----------


In [ ]:
name = '1935-09-30_manual_annotated.json'
with open(os.path.join('data/output4', name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
text_import

In [160]:
for (idx, row) in data[(data["filename"] == '1934-10-15_manual_annotated.json') & (data["sentence_id"] > 229) & (data["sentence_id"] < 235)].iterrows():
    print(row['sentence_id'], row['words'], row['labels'])

230 Kui O
230 seda O
230 maja O
230 linna O
230 asutiste O
230 jaoks O
230 kõigiti O
230 otstarbekohaseks O
230 muuta O
230 osutub O
230 paratamatuks O
230 ette O
230 võtta O
230 sisemisi O
230 ümberehitusi O
230 , O
230 samuti O
230 juure O
230 ehitada O
230 uusi O
230 ruume O
230 , O
230 milleks O
230 on O
230 olemas O
230 soodsad O
230 võimalused O
230 . O
231 Ümber- O
231 ja O
231 juureehituste O
231 kohta O
231 , O
231 kui O
231 see O
231 maja O
231 omandatakse O
231 linnale O
231 , O
231 koostatakse O
231 vastavad O
231 kavad O
231 ja O
231 kulude O
231 eelarved O
231 ja O
231 need O
231 esitatakseLinnavolikogule I-ORG
231 kinnitamiseks O
231 . O
232 Nimetatud O
232 maja O
232 linnale O
232 omandamise O
232 korral O
232 on O
232 võimalik O
232 katta O
232 ostuhinda O
232 ja O
232 ostuga O
232 seotud O
232 kulusid O
232 järgmiselt O
232 : O
232 1. O
233 Ostuhinnast O
233 lepingu O
233 sõlmimisel O
233 tasumisele O
233 kuuluvad O
233 kr O
233 . O
233 30.000 O
233 ja O
233 kinnistam

In [25]:
name = '1935-09-30_manual_annotated.json'
with open(os.path.join('output4', name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
for nr,i in enumerate(text_import.ne_gold_a):
    if i.text.startswith('Tamme'):
    #if i.text == "esitatakseLinnavolikogule":
    #if i.start == 39588:
        print(nr, i)
#text_import.words[6679].base_span 
text_import.tokens[0].text

370 Span('Tammeväljal', [{'tag': 'LOC'}])
500 Span('Tamme linnaosas', [{'tag': 'LOC'}])
510 Span('Tamme linnaosas', [{'tag': 'LOC'}])
735 Span('Tamme', [{'tag': 'LOC'}])
747 Span('Tamme linnaosa', [{'tag': 'LOC'}])
775 Span('Tamme l.o.', [{'tag': 'LOC'}])
794 Span('Tammelinnaosa', [{'tag': 'LOC'}])
822 Span('Tamme l.o.', [{'tag': 'LOC'}])


'Tartu'

In [33]:
name = '1935-09-30_manual_annotated.json'
with open(os.path.join('output4', name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
#print(text_import.sentences[231].start, text_import.sentences[231].end)
#print(text_import.ne_gold_a[364])
#sections = extract_sections(text_import, sections=[(54118,54128)])
#sections
#for nr, i in enumerate(text_import.ne_gold_a):
#    if i.text == "htusegümnaasiumi":
#        print(nr, i, i.start, i.end)
#print(text_import.ne_gold_a[642].start, text_import.ne_gold_a[642].text)
#print(text_import.ne_gold_a[747].start, text_import.ne_gold_a[747].end, text_import.ne_gold_a[747].text)
#print(text_import.sentences[668].start, text_import.sentences[668].end, text_import.sentences[668])
#print(text_import.words[11207])

64813 64827 Tamme linnaosa
66479 66707 EnvelopingSpan(['Linnavolikogu', ',', 'olles', 'asja', 'ettekande', 'ära', 'kuulanud', ',', 'lahtisel', 'hääletusel', 'ühel', 'häälel', 'otsustab', ':', '1', ')', 'Kinnitada', 'allpool', 'tähendatud', 'ehituskruntide', 'saajateks', 'järgmised', 'isikud', ':', 'a', ')', 'Tamme', 'linnaosa', 'maa-alal', ',', 'kinn', '.', 'nr.', '1', '(', 'IV', 'hüp', '.', 'ringkond', ')', ':', 'kvart', '.'], [{}])
Span('Tamme', [{'normalized_form': None}])


In [25]:
name = '1922-04-24_manual_annotated.json'
with open(os.path.join('output4', name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
print(len(text_import.sentences))
name = '1936-09-07_manual_annotated.json'
with open(os.path.join('output4', name), "r", encoding="utf-8") as file:
        content = file.read()
text_import = json_to_text(json_text=content)
print(len(text_import.sentences))

data = process_directory_to_dict('output4')
new = getData(data, ['1922-04-24_manual_annotated.json', '1936-09-07_manual_annotated.json'])
new[(new['sentence_id'] > 132) & (new['sentence_id'] < 135)]

133
461


,sentence_id,words,labels
43004,133,Juhatab,O
43005,133,Volikogu,O
43006,133,juhataja,O
43007,133,A.Piip,B-PER
43008,133,.,O
43009,134,Koos,O
43010,134,on,O
43011,134,33,O
43012,134,linnavolinikust,O
43013,134,25,O


In [111]:
from estnltk.taggers import TokenSplitter
import re

# Create a pattern that captures the components you want to split
pattern1 = re.compile(r'(?P<end>Tamme)linnaosamaa')

# Initialize the TokenSplitter with your pattern
token_splitter1 = TokenSplitter(patterns=[pattern1])

pattern2 = re.compile(r'(?P<end>linnaosa)maa')
token_splitter2 = TokenSplitter(patterns=[pattern2])

# Apply the splitter to your text
text = "asi Tammelinnaosamaa-alal asi2"
# You'd typically create a Text object from estnltk first
#from estnltk import Text
text_obj = Text(text)
text_obj.tag_layer('tokens')
print(text_obj.tokens.text)
token_splitter1.retag(text_obj)
print(text_obj.tokens.text)
token_splitter2.retag(text_obj)
print(text_obj.tokens.text)
preprocess_words( text_obj )
sentence_tokenizer.tag( text_obj )
postfix_sentence_breaks_inside_parentheses( text_obj, doc_name='' )
print(text_obj.words.text)

['asi', 'Tammelinnaosamaa', '-', 'alal', 'asi2']
['asi', 'Tamme', 'linnaosamaa', '-', 'alal', 'asi2']
['asi', 'Tamme', 'linnaosa', 'maa', '-', 'alal', 'asi2']
['asi', 'Tamme', 'linnaosa', 'maa-alal', 'asi2']


In [7]:
def stats():
    data = process_directory_to_dict('output4')
    for file in data['filename'].unique():
        print(
            file, "- Total:", str(len(data[(data["filename"] == file) & (data["labels"].str.startswith('B-'))])) + ";",
            "LOC:", str(len(data[(data["filename"] == file) & (data["labels"] == "B-LOC")])),
            "PER:", str(len(data[(data["filename"] == file) & (data["labels"] == "B-PER")])),
            "ORG:", str(len(data[(data["filename"] == file) & (data["labels"] == "B-ORG")])),
        )
stats()

1922-04-24_manual_annotated.json - Total: 166; LOC: 32 PER: 56 ORG: 78
1927-03-28_manual_annotated.json - Total: 495; LOC: 16 PER: 189 ORG: 290
1932-01-25_manual_annotated.json - Total: 723; LOC: 70 PER: 399 ORG: 254
1934-10-15_manual_annotated.json - Total: 527; LOC: 56 PER: 185 ORG: 286
1935-09-30_manual_annotated.json - Total: 562; LOC: 121 PER: 173 ORG: 268
1936-09-07_manual_annotated.json - Total: 323; LOC: 25 PER: 76 ORG: 222
1941-01-03_manual_annotated.json - Total: 37; LOC: 2 PER: 10 ORG: 25


In [9]:
groups = [['1922-04-24_manual_annotated.json', '1936-09-07_manual_annotated.json'], 
          ['1927-03-28_manual_annotated.json', '1941-01-03_manual_annotated.json'], 
          ['1932-01-25_manual_annotated.json'], 
          ['1934-10-15_manual_annotated.json'], 
          ['1935-09-30_manual_annotated.json']]

In [13]:
def getData(data, group):
    group_data = data[data['filename'].isin(group)].copy()
    if len(group) == 1:
        return group_data
    group_data = group_data.sort_values(by=['filename', 'sentence_id'])
    current_max_id = 0
    for file in group:
        group_data.loc[group_data['filename'] == file, 'sentence_id'] += current_max_id
        current_max_id = group_data[group_data['filename'] == file]['sentence_id'].max() + 1
    group_data.drop('filename', axis=1, inplace=True)
    return group_data

groups = [['1922-04-24_manual_annotated.json', '1936-09-07_manual_annotated.json'], 
          ['1927-03-28_manual_annotated.json', '1941-01-03_manual_annotated.json'], 
          ['1932-01-25_manual_annotated.json'], 
          ['1934-10-15_manual_annotated.json'], 
          ['1935-09-30_manual_annotated.json']]

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

labels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True
model = NERModel(
    "bert", "tartuNLP/EstBERT", args=model_args, labels=labels
)
data = process_directory_to_dict('data/output4')
start_time = time.time()

for eval_nr, eval_group in enumerate(groups):
    eval_data = getData(data, eval_group)
    
    all_train_files = []
    for i, group in enumerate(groups):
        if i != eval_nr:  # Skip the evaluation group
            all_train_files.extend(group)
    
    train_data = getData(data, all_train_files)
    model.train_model(train_data, eval_data=eval_data)
    result, model_outputs, preds_list = model.eval_model(eval_data)
    print(result)
    #break

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:ner_model.py:1884:  Converting to features started.


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Norman\anaconda3\Lib\site-packages\simpletransformers\ner\ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


...

C:\Users\Norman\anaconda3\Lib\site-packages\simpletransformers\ner\ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


INFO:ner_model.py:523:  Training of bert model complete. Saved to outputs/.
INFO:ner_model.py:1884:  Converting to features started.


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:ner_model.py:1253: {'eval_loss': 0.023045973866828717, 'precision': 0.9628975265017667, 'recall': 0.9732142857142857, 'f1_score': 0.9680284191829485}
{'eval_loss': 0.023045973866828717, 'precision': 0.9628975265017667, 'recall': 0.9732142857142857, 'f1_score': 0.9680284191829485}
Elapsed time: 161.2237 seconds
